In [5]:
! pip install langchain chromadb faiss-cpu langchain-community huggingface-hub wikipedia



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [6]:
from langchain_community.document_loaders import WikipediaLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document

In [7]:
loader = WikipediaLoader(query="Langchain",load_max_docs=2)
docs = loader.load()

In [8]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"   # you can change model if needed
)

/var/folders/gc/z45ptddn10j4dnt9brrqf50c0000gn/T/ipykernel_16276/1326109476.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [9]:
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory="./chroma_store"
)

query = "what is langchain"

results = vectordb.similarity_search(query,k=2)

for i,r in enumerate(results,1):
    print(f"\nResult{i}:")
    print(r.page_content[:500],"...")


Result1:
LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.


== History ==
LangChain was launched in October 2022 as an open source project by Harrison Chase, while working at machine learning startup Robust Intelligence. In April 2023,  ...

Result2:
LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.


== History ==
LangChain was launched in October 2022 as an open source project by Harrison Chase, while working at machine learning startup Robust Inte

### Vector Store Retriever

In [10]:
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [11]:
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_name="my_collection"
)

In [12]:
retriever = vectorstore.as_retriever(search_kwargs={"k":2})


In [13]:
query = "what is Chroma used for?"
results = retriever.invoke(query)

In [14]:
for i,doc in enumerate(results):
    print(f"\n--- Result{i+1}---")
    print(doc.page_content)


--- Result1---
Chroma is a vector database optimized for LLM-based search.

--- Result2---
LangChain helps developers build LLM applications easily.


In [15]:
results = vectorstore.similarity_search(query,k=2)

In [16]:
for i, doc in enumerate(results):
    print(f"\n---Results{i+1}---")
    print(doc.page_content)


---Results1---
Chroma is a vector database optimized for LLM-based search.

---Results2---
LangChain helps developers build LLM applications easily.


### MMR

In [17]:
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [18]:
from langchain_community.vectorstores import FAISS
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore = FAISS.from_documents(
    documents = docs,
    embedding=embedding_model
)

In [19]:
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k":3,"lambda_mult":0.5}
)

In [20]:
query = "what is langchain?"
results = retriever.invoke(query)

In [21]:
for i, doc in enumerate(results):
    print(f"\n---Result{i+1}---")
    print(doc.page_content)


---Result1---
LangChain supports Chroma, FAISS, Pinecone, and more.

---Result2---
LangChain is used to build LLM based applications.

---Result3---
Embeddings are vector representations of text.


### Multiquery Retriever

In [22]:
from langchain.retrievers.multi_query import MultiQueryRetriever

In [23]:
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [24]:
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

In [25]:
vectorstore = FAISS.from_documents(documents=all_docs,embedding=embedding_model)
similarity_retriever = vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":5})


In [39]:

from langchain_huggingface import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id="google/gemma-2-2b-it",
    task="text-generation"
)



In [40]:
from langchain.retrievers.multi_query import MultiQueryRetriever
multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(search_kwargs={"k":5}),
    llm = llm
)

In [41]:
query = "How to improve energy levels and maintain balance?"


In [43]:
similarity_results = similarity_retriever.invoke(query)
multiquery_results= multiquery_retriever.invoke(query)

HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/models/google/gemma-2-2b-it?expand=inferenceProviderMapping (Request ID: Root=1-68cfa473-6e24458972177941782f9634;c162b179-7fd3-46a0-a91a-ef185b1e55c3)

Invalid credentials in Authorization header

In [ ]:
for i, doc in enumerate(similarity_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

print("*"*150)

for i, doc in enumerate(multiquery_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

### ContextualCompressionRetriever

In [46]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor


In [47]:
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [48]:
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
vectorstore = FAISS.from_documents(docs, embedding_model)

In [49]:
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [44]:
llm = HuggingFaceEndpoint(
    repo_id="google/gemma-2-2b-it",
    task="text-generation"
)

In [50]:
compressor = LLMChainExtractor.from_llm(llm)

In [51]:
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

In [ ]:
# Query the retriever
query = "What is photosynthesis?"
compressed_results = compression_retriever.invoke(query)

In [ ]:
# Query the retriever
query = "What is photosynthesis?"
compressed_results = compression_retriever.invoke(query)